In [ ]:
# For Colab, update cuda version to prevent memory leak
# https://github.com/google/jax/issues/10949
!sudo apt-get -qq install libcudnn8=8.1.1.33-1+cuda11.2 cuda-11-2 -y --allow-change-held-packages

In [ ]:
!rm -rf jax-diffusion
!git clone https://github.com/andylolu2/jax-diffusion
%cd jax-diffusion
%pip install -q -r requirements.txt

In [ ]:
!XLA_PYTHON_CLIENT_MEM_FRACTION=0.9 python -m jax_diffusion.train --config jax_diffusion/configs/mnist.py --dry_run

In [ ]:
# for celeb_a
!scripts/build_celeb_a.sh

In [ ]:
# sample
import jax
from ml_collections import FrozenConfigDict

from jax_diffusion.configs.mnist import get_config as mnist_config
from jax_diffusion.train.trainer import Trainer

config = mnist_config()
# config.restore = ""
config = FrozenConfigDict(config)  # needed to be hashable

trainer = Trainer(jax.random.PRNGKey(0), **config.experiment_kwargs)
if config.restore is not None:
    trainer.restore_checkpoint(config.restore)

In [ ]:
import matplotlib.pyplot as plt

from jax_diffusion.utils.image import image_grid

img = trainer.sample(4, jax.random.PRNGKey(1))
img = image_grid(img)
plt.imshow()